# Counting Transcripts - Pipeline A

> This is part two of the Counting Transcripts series. We will cover the pipeline: Trimmomatic/STAR/cufflinks, a common sequence of tools to clean, map, and quantify the reads. We will use the data downloaded in [part 1]() of the series. This notebook should be located in the same folder as the previous notebooks.

## Setup

> As usual, we will first need to setup our working directory.

In [ ]:
###IMPORTANT: if you ever restart this notebook, you MUST rerun this code cell
import os

#feel free to change this location, because we will be working in this folder for the entire tutorial
os.environ['WORKDIR'] = './data' 

### Pipeline A: Trimmomatic/STAR/cufflinks

**1a.** [Trimmomatic](http://www.usadellab.org/cms/?page=trimmomatic) is a Java software to trim and remove low quality reads. It also removes adapter sequences from library preparation. After downloading the software from its site, extract the zip file and move it into the ~/software folder. Next, execute the following command:

In [ ]:
!java -jar ~/software/Trimmomatic-0.38/trimmomatic-0.38.jar PE -phred33 \
    $WORKDIR/FASTQ/SRR5511057_1.fastq $WORKDIR/FASTQ/SRR5511057_2.fastq \
    $WORKDIR/cleanFASTQ/SRR5511057.forward_paired.fastq $WORKDIR/cleanFASTQ/SRR5511057.forward_unpaired.fastq \
    $WORKDIR/cleanFASTQ/SRR5511057.reverse_paired.fastq $WORKDIR/cleanFASTQ/SRR5511057.reverse_unpaired.fastq \
    LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36

Let's look over the arguments used in this command. 
- `PE` stands for paired end, which is the type of sequencing our sample data came from. 
- `-phred33` [Phred](https://en.wikipedia.org/wiki/Phred_quality_score) + 33 Quality Score.
- `$WORKDIR/FASTQ/SRR5511057_1.fastq` `$WORKDIR/FASTQ/SRR5511057_2.fastq` is the relative location for our input data.
- `$WORKDIR/cleanFASTQ/SRR5511057.forward_paired.fastq` `$WORKDIR/cleanFASTQ/SRR5511057.forward_unpaired.fastq` is the relative location for our output forward paired and unpaired data, respectively.
- `$WORKDIR/cleanFASTQ/SRR5511057.reverse_paired.fastq` `$WORKDIR/cleanFASTQ/SRR5511057.reverse_unpaired.fastq` is the relative location for our output reverse paired and unpaired data, respectively.
- `LEADING:3` removes leading bases with a low quality (below quality 3).
- `TRAILING:3` removes trailing bases with a low quality (below quality 3).
- `SLIDINGWINDOW:4:15` scans the read with a 4-base sliding window, cutting when the average quality per base drops below 15.
- `MINLEN:36` drops reads below the 36 bases long.

DNA is double-stranded, but one strand is designated as the "forward strand" while the other strand is designated as the "reverse strand". Another common term for this is plus/minus strands. Since paired-end sequencing reads match both directions, the reads that are complementary to the forward strand are called "reverse reads", while the reads that are complementary with the reverse strand are called "forward reads". 

In paired-end sequencing, reads come in pairs (each fragment yields two reads, one from each end), so if one of the reads is trimmed off by the criteria we set when running Trimmomatic, Trimmomatic will automatically remove the corresponding read and label both reads as "unpaired".
***

**1b.** [Star](https://github.com/alexdobin/STAR) (Spliced Transcripts Alignment to a Reference) is a C++ program to align reads to a reference genome. For performance efficiency, its first step is building an index of our reference genome (this step only needs to be done once!).

First, we will need to download STAR software and untar the file. Afterwards, remember to place STAR in the ~/software folder and add it to your PATH.

In [ ]:
!wget https://github.com/alexdobin/STAR/archive/2.7.0d.tar.gz
!tar -xzf 2.7.0d.tar.gz

**1c.** The first step to mapping our reads is to generate an index. Since we are interested in the expression of transcripts, we will download the reference transcriptome sequence from [MicrobesOnline](http://www.microbesonline.org/), an online database of microbial genomic information. Our data was generated in a sample from yeast, or scientifically known as *Saccharomyces ceravisiae*, so we need the FASTA transcriptome for yeast from [here](http://www.microbesonline.org/cgi-bin/genomeInfo.cgi?tId=4932). Unzip the file, and move the resulting file to the $WORKDIR/microbesOnline folder. Then, we will need to make a new folder to store the transcriptome indices.

In [ ]:
!mkdir $WORKDIR/transcriptomeIndex

To build the index, we will run the following command.

In [ ]:
!STAR --runThreadN 8 --runMode genomeGenerate \
    --genomeDir $WORKDIR/transcriptomeIndex --genomeFastaFiles $WORKDIR/microbesOnline/4932.transcriptomes.fasta

Each -- defines a new parameter, which are options we specify to STAR
- `--runThreadN 8` specifies the number of threads on your computer (hint: run `sysctl -n hw.ncpu` to find out!).
- `--runMode genomeGenerate` means that we are building the genome index.
- `--genomeDir $WORKDIR/transcriptomeIndex` saves the output the the folder we just created.
- `--genomeFastaFiles` are the reference genome sequences (in this case we are using the transcriptome) we are building our index from.

**1d.** Now, we map our reads to the transcriptome!

In [ ]:
!STAR --runThreadN 4 --genomeDir $WORKDIR/transcriptomeIndex \
    --readFilesIn $WORKDIR/cleanFASTQ/SRR5511057.forward_paired.fastq $WORKDIR/cleanFASTQ/SRR5511057.reverse_paired.fastq \
     --outSAMtype BAM SortedByCoordinate --outFileNamePrefix $WORKDIR/BAM/


If you ran into this error: 

`
BAMoutput.cpp:27:BAMoutput: exiting because of *OUTPUT FILE* error: could not create output file ./data/BAM_STARtmp//BAMsort/4/48`

SOLUTION: check that the path exists and you have write permission for this file. Also check `ulimit -n` and increase it to allow more open files. Also, it may be worth lowering the number of threads or running `ulimit -n 10000` in the terminal.
***

Let's look at the options used in this command:
- `--runThreadN 4` specifies the number of threads you want to use (run `sysctl -n hw.ncpu` to find the number of threads on your computer!).
- `--genomeDir $WORKDIR/transcriptomeIndex` directory where genome index was generated.
- `--readFilesIn` input sequence data to be mapped.
- `--outSAMtype BAM SortedByCoordinate` outputs a BAM file with sorted aligned reads.
- `--outFileNamePrefix` prefix to add to the output file.

Our aligned reads are stored in a BAM format, which is a compressed binary version of [SAM](https://en.wikipedia.org/wiki/SAM_(file_format)), a text-based format for storing aligned reads.

**1e.** We will now use [cufflinks](http://cole-trapnell-lab.github.io/cufflinks/) to quantify the reads per feature. Download the latest version [here](http://cole-trapnell-lab.github.io/cufflinks/install/), and add the directory to your Path. 

In [ ]:
!cufflinks $WORKDIR/BAM/Aligned.sortedByCoord.out.bam --library-type fr-firststrand

- `$WORKDIR/BAM/Aligned.sortedByCoord.out.bam` is the file of RNA-seq read alignments in BAM or SAM format
- `--library-type fr-firststrand` specifies the library used to prepare our data, we used Illumina's stranded Tru-Seq which is fr-firststrand

Cufflinks should generate 4 files: genes.fpkm_tracking, isoforms.fpkm_tracking, skipped.gtf, and transcripts.gtf. Move these files into your $WORKDIR/cufflinks folder. 

We are interested in the genes.fpkm_tracking file. In our example, we used the FASTA transcriptome to build the STAR index, so the genes.fpkm_tracking file quantifies the expression of every transcript. However, if we were to use the FASTA genome and a GFF3 annotation file to build the STAR index, genes.fpkm_tracking would quantify gene expression while isoforms.fpkm_tracking would quantify the expression for all possible exons.

You can open genes.fpkm_tracking using any text editor. You should have a table that looks like the following:

|tracking_id	| class_code | nearest_ref_id | gene_id | gene_short_name | tss_id | locus | length | coverage | FPKM | FPKM_conf_lo | FPKM_conf_hi | FPKM_status |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| CUFF.1 | - | - | CUFF.1 | - | - | VIMSS6580636:30-1540 | - | - | 3.54675 | 2.207 | 4.8865 | OK |
| CUFF.2 | - | - | CUFF.2 | - | - | VIMSS6580642:3238-3969 | - | - | 19.8471 | 14.9253 | 24.769 | OK |
| ... |

In this tutorial, we only care about a few of the columns:
1. `gene_id` is the Cufflinks gene id
2. `locus` is where the reads was aligned to on the transcriptome, for example: VIMSS6580636:30-1540 means on VIMSS6580636 transcript from the 30th to the 1540th base pair
3. `FPKM` is level of transcript expression in FPKM
4. `FPKM_conf_lo` is the lower bound of the 95% confidence interval for FPKM
5. `FPKM_conf_hi` is the upper bound of the 95% confidence interval for FPKM
6. `FPKM_status` is fairly self-explanatory

By looking at the FPKM value, we can gain a general understanding of the level of expression of each transcript:
- 0-10 is considered low
- 10-100 is considered medium
- 100-1000 is considered high
- 1000+ is considered very high

## Check Your Understanding

1. If we used pipeline A to measure gene expression in humans and built the STAR index using the entire human genome and an annotation file, would the genes.fpkm_tracking file or isoforms.fpkm_tracking file have more entries?

## Answers

1. We would expect the isoforms.fpkm_tracking file to contain more entries than the genes.fpkm_tracking file, because while humans only have around 25,000 genes, due to alternative splicing, we have a greater number of exons in our transcriptome than number of genes.